In [3]:
import pandas as pd

In [4]:
train=pd.read_csv("D:/my_work/python_codes/git_projects/car_insurance_claim_prediction/dataset/train.csv")
test_x=pd.read_csv("D:/my_work/python_codes/git_projects/car_insurance_claim_prediction/dataset/test.csv")
test_y=pd.read_csv("D:/my_work/python_codes/git_projects/car_insurance_claim_prediction/dataset/sample_submission.csv")
test=pd.concat([test_x,test_y.iloc[:,-1]],axis=1)
data=pd.concat([train,test],ignore_index=True)

In [5]:
data["is_claim"].value_counts(normalize=True)

0    0.56161
1    0.43839
Name: is_claim, dtype: float64

Little skew, but ok. Manageable

In [6]:
data.isnull().sum()

policy_id                           0
policy_tenure                       0
age_of_car                          0
age_of_policyholder                 0
area_cluster                        0
population_density                  0
make                                0
segment                             0
model                               0
fuel_type                           0
max_torque                          0
max_power                           0
engine_type                         0
airbags                             0
is_esc                              0
is_adjustable_steering              0
is_tpms                             0
is_parking_sensors                  0
is_parking_camera                   0
rear_brakes_type                    0
displacement                        0
cylinder                            0
transmission_type                   0
gear_box                            0
steering_type                       0
turning_radius                      0
length      

no null values. But have to manage future null values

In [7]:
data_x=data.drop(["policy_id","is_claim"],axis=1)
data_y=data["is_claim"]

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data_x,data_y,test_size=0.2,random_state=42)

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler,OrdinalEncoder
num_type=x_train.select_dtypes(include="number").columns
cat_type=x_train.select_dtypes(include="object").columns

pre_processor=ColumnTransformer(transformers=[
        ("numeric",MinMaxScaler(),num_type),
        ("categorical",OrdinalEncoder(),cat_type)
])

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

pipeline=Pipeline(steps=[
    ("preprocessor",pre_processor),
    ("model",DecisionTreeClassifier())
])

In [16]:
pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric', MinMaxScaler(),
                                                  Index(['policy_tenure', 'age_of_car', 'age_of_policyholder',
       'population_density', 'make', 'airbags', 'displacement', 'cylinder',
       'gear_box', 'turning_radius', 'length', 'width', 'height',
       'gross_weight', 'ncap_rating'],
      dtype='object')),
                                                 ('categorical',
                                                  OrdinalEncoder(),
                                                  Index...
       'rear_brakes_type', 'transmission_type', 'steering_type',
       'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer',
       'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks',
       'is_central_locking', 'is_power_steering',
       'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror',
       'is_ecw', 'is_speed_alert'],
      dtype='object'))])),
                ('model', DecisionTreeClassifier())])

In [18]:
pred=pipeline.predict(x_test)
from sklearn.metrics import classification_report,f1_score,confusion_matrix
print(f1_score(y_test,pred))

print(classification_report(y_test,pred))

print(confusion_matrix(y_test,pred))

0.4427436655650258
              precision    recall  f1-score   support

           0       0.57      0.55      0.56     11020
           1       0.44      0.45      0.44      8511

    accuracy                           0.51     19531
   macro avg       0.50      0.50      0.50     19531
weighted avg       0.51      0.51      0.51     19531

[[6102 4918]
 [4693 3818]]


In [19]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
smt=SMOTE(random_state=42)
pipeline = Pipeline(steps=[('preprocessor', pre_processor),
                            ('smote', smt),
                            ('model', DecisionTreeClassifier())
                          ])

pipeline.fit(x_train,y_train)

pred=pipeline.predict(x_test)
print(f1_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.4400462962962963
              precision    recall  f1-score   support

           0       0.56      0.55      0.56     11020
           1       0.43      0.45      0.44      8511

    accuracy                           0.50     19531
   macro avg       0.50      0.50      0.50     19531
weighted avg       0.51      0.50      0.51     19531

[[6053 4967]
 [4709 3802]]
